In [168]:
import Pkg; Pkg.add("BenchmarkTools")

   Resolving package versions...
   Installed BenchmarkTools ─ v1.5.0
    Updating `/spin1/home/linux/saakyanh2/.julia/environments/v1.11/Project.toml`
  [6e4b80f9] + BenchmarkTools v1.5.0
    Updating `/spin1/home/linux/saakyanh2/.julia/environments/v1.11/Manifest.toml`
  [6e4b80f9] + BenchmarkTools v1.5.0
  [9abbd945] + Profile v1.11.0
Precompiling project...
   1267.3 ms  ✓ BenchmarkTools
  1 dependency successfully precompiled in 3 seconds. 215 already precompiled.


In [169]:
using Distances
using BenchmarkTools

In [ ]:
include("foldexity.jl")

In [333]:
pdb = readpdb("testpdb/3EFM.pdb")
writepdb(pdb, "testpdb/backbone.pdb")
fsize = 8

xyzcoords = pdb2matrix(pdb)
coordmatrix = matrix2fragments(xyzcoords, fsize)

pdbmatrix = pdb2pdbmatrix(pdb)
pdbfragments = matrix2fragments(pdbmatrix, fsize)

println()


In [334]:
# dump PDB fragments

pdb = readpdb("testpdb/small.pdb")
pdbfragments = matrix2fragments(pdbmatrix, 20)
for (i, frag) in enumerate(pdbfragments)
    writepdb(pdbmatrix2pdb(frag), "frags/frag$i.pdb")
end

In [341]:
pfrag1 = coordmatrix[2]
pfrag2 = coordmatrix[1]

@btime align_rmsd(pfrag1, pfrag2)
@btime align_rmsd2(pfrag1, pfrag2)

  7.166 μs (170 allocations: 10.66 KiB)
  5.667 μs (76 allocations: 6.88 KiB)


8.284607f0

In [340]:
function align_rmsd(m1,m2)

    normalisedP = (translate_to_centroid(m1))
    normalisedQ = (translate_to_centroid(m2))

    xcov = cross_covariance_matrix(normalisedP,normalisedQ)
    orot = optimal_rotation_matrix(xcov)

    num_atoms = size(normalisedP)[1]
    rotated = zeros(Float64,num_atoms,3)

    for i=1:num_atoms
        rotated[i,:] = orot*normalisedP[i,:]
    end

    RMSD_value = norm(rotated-normalisedQ)

    #TMscore
    distances = colwise(euclidean, rotated', normalisedQ')
    L0 = num_atoms
    d0 = 1.24 * cbrt(fsize - 15) - 1.8
    tm = (1/L0) * sum([1 / (1 + ((d/d0) ^2)) for d in distances])

    return RMSD_value
end


function align_rmsd2(m1,m2)

    normalisedP = (translate_to_centroid(m1))
    normalisedQ = (translate_to_centroid(m2))

    xcov = cross_covariance_matrix(normalisedP,normalisedQ)
    orot = optimal_rotation_matrix(xcov)

    num_atoms = size(normalisedP)[1]
    rotated = zeros(Float64,num_atoms,3)

    rotated  = (orot*normalisedP')'

    RMSD_value = norm(rotated-normalisedQ)

    #TMscore
    distances = colwise(euclidean, rotated', normalisedQ')
    L0 = num_atoms
    d0 = 1.24 * cbrt(fsize - 15) - 1.8
    tm = (1/L0) * sum([1 / (1 + ((d/d0) ^2)) for d in distances])
    
    return RMSD_value
end


align_rmsd2 (generic function with 1 method)

In [304]:
#TMscore
distances = colwise(euclidean, rotated', normalisedQ')
L0 = fsize*3
d0 = 1.24 * cbrt(fsize - 15) - 1.8
(1/L0) * sum([1 / (1 + ((d/d0) ^2)) for d in distances])


0.004720466371910841

In [4]:
aver_rmsd, nclusts, fxity, nres, matrix = fxity_kabsh(coordmatrix)

(12.604134669018569, 37, 0.9487179487179487, 39, [0.0 15.259918127243338 … 15.673347910584875 6.248241906285712; 15.259918127243338 0.0 … 17.808255124041203 14.988237305484544; … ; 15.673347910584875 17.808255124041203 … 0.0 14.628995374500345; 6.248241906285712 14.988237305484544 … 14.628995374500345 0.0])

In [5]:
fxity, nfrags = fxpdb("testpdb/small.pdb") 
data = fxdir("testpdb/", "tmp", 4, true)  
rm("tmp")

Starting foldexity ...
5 files are collected 
Starting foldexity ...
1	testpdb/3EFM.pdb	0.5202108963093146	296	4.470485886057379	569
2	testpdb/AF-Q8VCK6-F1-model_v4.pdb	0.26605504587155965	87	2.6608642385790833	327
3	testpdb/polya.pdb	0.1724137931034483	10	0.364438980618678	58
4	testpdb/small.pdb	0.2978723404255319	14	2.208886731628906	47
5	testpdb/small_backbone.pdb	0.2978723404255319	14	2.208886731628906	47
